In [ ]:
import pandas as pd
from datetime import datetime, time, timedelta

# Load dataset
df = pd.read_csv("/content/dataset.csv")

# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00


In [ ]:
# Show available locations
available_locations = df['SystemCodeNumber'].unique()
print("Available Location:", available_locations)

# Ask user for LocationID
location_id = input("Enter Location from the above list: ").strip()

# Validate input
if location_id not in df['SystemCodeNumber'].unique():
    raise ValueError("Invalid Location. Please choose from the available list.")

# Filter dataset for that location
df_location = df[df['SystemCodeNumber'] == location_id].copy()

Available Location: ['BHMBCCMKT01' 'BHMBCCTHL01' 'BHMEURBRD01' 'BHMMBMMBX01' 'BHMNCPHST01'
 'BHMNCPNST01' 'Broad Street' 'Others-CCCPS105a' 'Others-CCCPS119a'
 'Others-CCCPS135a' 'Others-CCCPS202' 'Others-CCCPS8' 'Others-CCCPS98'
 'Shopping']
Enter Location from the above list: BHMEURBRD01


In [ ]:
# Define base time (8:00 AM) and 30-minute intervals till 4:30 PM
base_time = datetime.strptime("08:00", "%H:%M").time()
time_slots = [(datetime.combine(datetime.today(), base_time) + timedelta(minutes=30 * i)).time() for i in range(18)]

# Set price model parameters
base_price = 10.0
alpha_min = 0.5
alpha_max = 3.0

# Initialize tracking
results = []
prev_occ = None
prev_cap = None
prev_price = base_price


In [ ]:
# Process each half-hour time slot
for slot in time_slots:
    # ±5 minute window for averaging
    dt_slot = datetime.combine(datetime.today(), slot)
    start_window = (dt_slot - timedelta(minutes=5)).time()
    end_window = (dt_slot + timedelta(minutes=5)).time()

    # Filter data within window using the time component of the 'Timestamp' column
    slot_data = df_location[(df_location['Timestamp'].dt.time >= start_window) & (df_location['Timestamp'].dt.time <= end_window)]

    if slot_data.empty:
        continue

    # Calculate averages
    avg_occ = slot_data['Occupancy'].mean()
    avg_cap = slot_data['Capacity'].mean()

    # Compute dynamic alpha and price
    if avg_cap == 0:
        alpha_t = 0
        price = prev_price
    else:
        if prev_occ is None:
            alpha_t = None
            price = prev_price
        else:
            if avg_occ >= prev_occ:
                load_ratio = avg_occ / avg_cap
                alpha_t = alpha_min + (alpha_max - alpha_min) * load_ratio
            else:
                load_ratio = prev_occ / prev_cap if prev_cap != 0 else 0
                alpha_t = -1 * (alpha_min + (alpha_max - alpha_min) * load_ratio)

            price = prev_price + alpha_t * load_ratio

    # Append row to results
    results.append({
        "Location": location_id,
        "TimeSlot": slot.strftime("%H:%M"),
        "AvgOccupancy": round(avg_occ, 2),
        "Capacity": int(avg_cap),
        "Alpha": round(alpha_t, 3) if alpha_t is not None else None,
        "Price": round(price, 2)
    })

    # Update tracking values
    prev_occ = avg_occ
    prev_cap = avg_cap
    prev_price = price

In [ ]:
# Create final DataFrame
result_df = pd.DataFrame(results)

# Save to CSV
output_path = f"/content/location_{location_id}_pricing.csv"
result_df.to_csv(output_path, index=False)

# Show the result
print(f"\nDynamic pricing results for LocationID {location_id}:\n")
print(result_df)



Dynamic pricing results for LocationID BHMEURBRD01:

       Location TimeSlot  AvgOccupancy  Capacity  Alpha  Price
0   BHMEURBRD01    08:00        112.22       470    NaN  10.00
1   BHMEURBRD01    08:30        159.63       470  1.349  10.46
2   BHMEURBRD01    09:00        220.17       470  1.671  11.24
3   BHMEURBRD01    09:30        253.54       470  1.849  12.24
4   BHMEURBRD01    10:00        288.34       470  2.034  13.49
5   BHMEURBRD01    10:30        309.11       470  2.144  14.90
6   BHMEURBRD01    11:00        324.25       470  2.225  16.43
7   BHMEURBRD01    11:30        334.73       470  2.280  18.06
8   BHMEURBRD01    12:00        352.27       470  2.374  19.83
9   BHMEURBRD01    12:30        350.23       470 -2.374  18.06
10  BHMEURBRD01    13:00        375.07       470  2.495  20.05
11  BHMEURBRD01    13:30        356.67       470 -2.495  18.06
12  BHMEURBRD01    14:00        358.10       470  2.405  19.89
13  BHMEURBRD01    14:30        361.32       470  2.422  21.75
1

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import push_notebook

# Enable Bokeh output in notebooks
output_notebook()

# Prepare data
source = ColumnDataSource(data={
    'TimeSlot': result_df['TimeSlot'],
    'Price': result_df['Price'],
    'Alpha': result_df['Alpha'],
    'AvgOccupancy': result_df['AvgOccupancy'],
    'Capacity': result_df['Capacity']
})

# Create figure
p = figure(x_range=result_df['TimeSlot'],
           title=f"Dynamic Parking Price for Location: {location_id}",
           x_axis_label='Time Slot',
           y_axis_label='Price ($)',
           width=800,
           height=400,
           toolbar_location="above",
           tools="pan,wheel_zoom,box_zoom,reset,save")

# Add line and circle markers
p.line(x='TimeSlot', y='Price', source=source, line_width=2, color='navy', legend_label="Price")
p.scatter(x='TimeSlot', y='Price', source=source, size=6, color='orange', legend_label="Price Points")

# Add hover tooltip
hover = HoverTool(tooltips=[
    ("Time Slot", "@TimeSlot"),
    ("Price ($)", "@Price"),
    ("Alpha", "@Alpha"),
    ("Occupancy", "@AvgOccupancy"),
    ("Capacity", "@Capacity")
])
p.add_tools(hover)

# Improve readability
p.xaxis.major_label_orientation = 1.0  # Rotate x-axis labels
p.legend.location = "top_left"

# Show the plot
show(p)